# Etape 1.1 : Exploration initiale

## Charger les donnees de consommation avec PySpark

In [50]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import os

# Chemins de toutes les donnees
DATA_DIR = "../data_ecf"

CONSUMPTION_PATH = os.path.join(DATA_DIR, "consommations_raw.csv")

# Création de la session Spark 

spark = SparkSession.builder \
    .appName("ECF2") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

# Reduire les logs
spark.sparkContext.setLogLevel("WARN")

print(f"Spark version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")


Spark version: 4.1.1
Spark UI: http://host.docker.internal:4040


## Analyser le schema infere et identifier les problemes de typage

In [51]:
# Charger le CSV avec inference de schema
df_consumption = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(CONSUMPTION_PATH)

print(f"Nombre de lignes: {df_consumption.count():,}")
print(f"Nombre de colonnes: {len(df_consumption.columns)}")

Nombre de lignes: 7,758,868
Nombre de colonnes: 5


In [52]:
# Afficher le schema infere
print("Schema infere:")
df_consumption.printSchema()

Schema infere:
root
 |-- batiment_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- type_energie: string (nullable = true)
 |-- consommation: string (nullable = true)
 |-- unite: string (nullable = true)



In [53]:
# PBS DE TYPAGE : 

# Le timestamp est en string et le format n'est pas toujours le même

print("Exemples de formats de timestamp:")
df_consumption.select("timestamp").distinct().show(20, truncate=False)

# L'unité n'est pas toujours la même (m3,kWh,...)

# La consommation est en string : elle peut contenir des virgules et des guillemets, null, N/A, erreur

df_non_numeric = df_consumption.filter(
    ~F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")
)

print(f"Nombre de valeurs non numeriques: {df_non_numeric.count():,}")
df_non_numeric.select("consommation").distinct().show()

# Valeurs avec virgule comme separateur decimal et guillemets
df_with_comma = df_consumption.filter(F.col("consommation").contains(","))
print(f"Nombre de valeurs avec virgule: {df_with_comma.count():,}")
df_with_comma.select("consommation").show(5)


Exemples de formats de timestamp:
+-------------------+
|timestamp          |
+-------------------+
|2023-12-21 13:00:00|
|11/29/2024 04:00:00|
|09/15/2024 18:00:00|
|20/12/2023 07:00   |
|29/04/2024 13:00   |
|2024-06-15T14:00:00|
|22/03/2024 11:00   |
|2023-04-07 13:00:00|
|2024-03-11T11:00:00|
|08/21/2024 17:00:00|
|2024-05-27 04:00:00|
|08/02/2023 16:00   |
|2024-11-23 17:00:00|
|2023-07-06T05:00:00|
|2024-04-30 02:00:00|
|08/12/2023 06:00   |
|16/11/2024 21:00   |
|12/08/2024 04:00:00|
|2024-05-27 10:00:00|
|11/03/2023 09:00:00|
+-------------------+
only showing top 20 rows
Nombre de valeurs non numeriques: 38,975
+------------+
|consommation|
+------------+
|        null|
|         N/A|
|      erreur|
|         ---|
+------------+

Nombre de valeurs avec virgule: 925,392
+------------+
|consommation|
+------------+
|       10,10|
|       72,29|
|       17,82|
|        0,92|
|      290,65|
+------------+
only showing top 5 rows


## Calculer les statistiques descriptives par type d'energie

In [54]:
# Convertir consommation en double (en remplacant la virgule par un point)
#utilisation de try catch pour correspondance avec vesrion3.4/3.5 de spark
df_consumption_numeric = df_consumption.withColumn(
    "conso_clean",
    F.expr("try_cast(regexp_replace(cast(consommation as string), ',', '.') as double)")
)

# Statistiques par polluant (en ignorant les valeurs nulles)
stats_by_energy_type = df_consumption_numeric.filter(F.col("conso_clean").isNotNull()) \
    .groupBy("type_energie") \
    .agg(
        F.count("*").alias("count"),
        F.round(F.mean("conso_clean"), 2).alias("mean"),
        F.round(F.stddev("conso_clean"), 2).alias("stddev"),
        F.round(F.min("conso_clean"), 2).alias("min"),
        F.round(F.max("conso_clean"), 2).alias("max"),
        F.round(F.expr("percentile(conso_clean, 0.5)"), 2).alias("median")
    ) \
    .orderBy("type_energie")

print("Statistiques par type d'energie:")
stats_by_energy_type.show()

Statistiques par type d'energie:
+------------+-------+------+-------+--------+--------+------+
|type_energie|  count|  mean| stddev|     min|     max|median|
+------------+-------+------+-------+--------+--------+------+
|         eau|2573156|204.36|2398.57| -657.01|49999.23|  7.52|
| electricite|2573364|430.64|2429.51|-4003.35|49999.13|108.56|
|         gaz|2573373|560.94|2465.81|-5963.49|49999.49|160.57|
+------------+-------+------+-------+--------+--------+------+



In [ ]:
# Identifier les valeurs aberrantes
print("Valeurs negatives:")
df_consumption_numeric.filter(F.col("conso_clean") < 0).show(5)

print("\nValeurs > 10000 :")
df_consumption_numeric.filter(F.col("conso_clean") > 10000).groupBy("type_energie").count().show()

Valeurs negatives:
+-----------+-------------------+------------+------------+-----+-----------+
|batiment_id|          timestamp|type_energie|consommation|unite|conso_clean|
+-----------+-------------------+------------+------------+-----+-----------+
|    BAT0075|01/14/2023 08:00:00|         eau|       -6.64|   m3|      -6.64|
|    BAT0027|08/31/2023 08:00:00| electricite|     -115.03|  kWh|    -115.03|
|    BAT0138|2023-02-13T20:00:00| electricite|     -144.92|  kWh|    -144.92|
|    BAT0023|2023-07-17T11:00:00| electricite|     -175.03|  kWh|    -175.03|
|    BAT0022|2024-12-04 11:00:00|         eau|       -8.94|   m3|      -8.94|
+-----------+-------------------+------------+------------+-----+-----------+
only showing top 5 rows

Valeurs > 1000 ug/m3:
+------------+-----+
|type_energie|count|
+------------+-----+
|         eau|12810|
|         gaz|12764|
| electricite|12986|
+------------+-----+



## Identifier les bâtiments avec le plus de mesures

In [57]:

# Nombre d'enregistrements par batiments
records_by_building = df_consumption_numeric.groupBy("batiment_id") \
    .count() \
    .orderBy(F.desc("count"))

print("Top 10 bâtiments avec le plus de mesures:")
records_by_building.show(10)

Top 10 bâtiments avec le plus de mesures:
+-----------+-----+
|batiment_id|count|
+-----------+-----+
|    BAT0086|53275|
|    BAT0002|53257|
|    BAT0145|53255|
|    BAT0117|53254|
|    BAT0047|53254|
|    BAT0051|53246|
|    BAT0093|53242|
|    BAT0078|53235|
|    BAT0146|53235|
|    BAT0046|53233|
+-----------+-----+
only showing top 10 rows


## Produire un rapport d'audit de qualite des donnees

In [60]:
# Resume des problemes

total = df_consumption.count()

# Valeurs non numeriques
non_numeric = df_consumption.filter(
    ~F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")
).count()

# Consommations avec virgule
with_comma = df_consumption.filter(F.col("consommation").contains(",")).count()

# Valeurs negatives (apres conversion)
negative = df_consumption_numeric.filter(F.col("conso_clean") < 0).count()

# Valeurs aberrantes > 10000
outliers = df_consumption_numeric.filter(F.col("conso_clean") > 10000).count()

# Doublons
duplicates = total - df_consumption.dropDuplicates(["batiment_id", "timestamp", "type_energie"]).count()


print(f"Total enregistrements: {total:,}")
print()
print(f"Problemes identifies:")
print(f"  - Valeurs non numeriques: {non_numeric:,} ({non_numeric/total*100:.2f}%)")
print(f"  - Valeurs avec virgule decimale: {with_comma:,} ({with_comma/total*100:.2f}%)")
print(f"  - Valeurs negatives: {negative:,} ({negative/total*100:.2f}%)")
print(f"  - Valeurs aberrantes (>10000): {outliers:,} ({outliers/total*100:.2f}%)")
print(f"  - Doublons: {duplicates:,} ({duplicates/total*100:.2f}%)")
print(f"  - Formats de dates multiples")

Total enregistrements: 7,758,868

Problemes identifies:
  - Valeurs non numeriques: 38,975 (0.50%)
  - Valeurs avec virgule decimale: 925,392 (11.93%)
  - Valeurs negatives: 38,910 (0.50%)
  - Valeurs aberrantes (>10000): 38,560 (0.50%)
  - Doublons: 152,134 (1.96%)
  - Formats de dates multiples
